# GENERATE DATA

OLD ERRORAS WHEN USING JOBLIB(works single type on python 3.7.12 and multi one is throwing thrashing kinda error

with base 3.12.4 no multi error but inhomogenous librosa error idk y
)


WORKING ON PYTHON 3.7.12


In [ ]:
from preprocessing.brewData import brew

brew()

In [1]:
from preprocessing.dataReadFromDisk import *
df = load_df_from_csv()
# df = df[df["total_length"]<120]
df = df[(df["BeatDivisor"]==4) & (df["total_length"]<300)]
# df=df[:500]
df

DataFrame loaded from: E:\Notebooks\melokai\processed-metadata\processed-metadata.csv


,audio,split,folder,beatmapset_id,beatmap_id,approved,total_length,hit_length,version,file_md5,...,difficultyrating,StackLeniency,DistanceSpacing,BeatDivisor,HPDrainRate,CircleSize,OverallDifficulty,ApproachRate,SliderTickRate,SliderMultiplier
0,00010f0eb02ee131aacac54bf72d5444,train,5bea04ca275826768fa4f8411822c388,830266,1739583,1,221,215,Zero Vector,00010f0eb02ee131aacac54bf72d5444,...,4.93655,0.0,1.20,4,5.0,5,8.0,8.0,1,1.2
1,000504bce087e1e44a54862f311ea66a,train,37d8553a6384d8c65608eed50013f424,1115124,2563178,1,127,89,Extra,000504bce087e1e44a54862f311ea66a,...,5.63927,0.4,1.25,4,5.5,4,8.4,9.2,1,1.5


In [2]:
input_audios,processed_beatmap_list,hyper_params_list = new_process_audio_and_beatmap_for_model_all_and_save(df)

0 = nothing, 1=circle, 2= slider start, 3=slider mid and end, 4=spinner start and end(fix), 5= spinner mid

In [3]:
for o in processed_beatmap_list[1]:
    print(o)

[  0   0 286   0]
[  0   0 365   0]
[  0   0 444   0]
[  0   0 523   0]
[  0   0 602   0]
[  0   0 681   0]
[  0   0 760   0]
[  0   0 839   0]
[  0   0 918   0]
[  0   0 997   0]
[   0    0 1076    0]
[   0    0 1155    0]
[   0    0 1234    0]
[   0    0 1313    0]
[   0    0 1392    0]
[   0    0 1471    0]
[   0    0 1549    0]
[   0    0 1628    0]
[   0    0 1707    0]
[   0    0 1786    0]
[   0    0 1865    0]
[   0    0 1944    0]
[   0    0 2023    0]
[   0    0 2102    0]
[   0    0 2181    0]
[   0    0 2260    0]
[   0    0 2339    0]
[   0    0 2418    0]
[   0    0 2497    0]
[   0    0 2576    0]
[   0    0 2655    0]
[   0    0 2734    0]
[   0    0 2813    0]
[   0    0 2892    0]
[   0    0 2971    0]
[   0    0 3049    0]
[   0    0 3128    0]
[   0    0 3207    0]
[   0    0 3286    0]
[   0    0 3365    0]
[   0    0 3444    0]
[   0    0 3523    0]
[   0    0 3602    0]
[   0    0 3681    0]
[   0    0 3760    0]
[   0    0 3839    0]
[   0    0 3918    0]
[   0 